# Understanding Cumulants and Their Application

**Cumulants** are a set of statistical properties of a probability distribution that, like moments, describe the shape of the distribution. However, unlike moments, cumulants possess the property of additivity: the cumulant of a sum of independent random variables is the sum of their individual cumulants. This makes them ideal for revealing **nonlinear and high-order interactions** between variables.

---

## Mathematical Definition of Cumulants

Let $X_1, X_2, \ldots, X_N$ be a set of random variables (in your case, the binarized activity from different EEG channels). Let $\tilde{X}_i = X_i - E[X_i]$ denote the mean-centered variable for $X_i$.

1.  **First Cumulant ($C_1$) - The Mean:**
    For a single variable $X_i$:
    $$C_1(X_i) = E[X_i]$$
    This represents the **expected value** or **mean** of the variable. In binary data, it's essentially the probability of the channel being active.

2.  **Second Cumulant ($C_2$) - The Covariance:**
    For a pair of variables $X_i, X_j$:
    $$C_2(X_i, X_j) = E[\tilde{X}_i\tilde{X}_j]$$
    This represents the **covariance** between $X_i$ and $X_j$. If $i=j$, it's the **variance** of $X_i$. It describes second-order (linear) interactions.

3.  **Third Cumulant ($C_3$) - Related to Skewness:**
    For a triplet of variables $X_i, X_j, X_k$:
    $$C_3(X_i, X_j, X_k) = E[\tilde{X}_i\tilde{X}_j\tilde{X}_k]$$
    This represents the **third-order interaction** among the variables. It's the third central moment. A non-zero value indicates **asymmetry** or skewness in the joint distribution and suggests the presence of interactions beyond the second order.

4.  **Fourth Cumulant ($C_4$) - Related to Kurtosis:**
    For a quadruplet of variables $X_i, X_j, X_k, X_l$:
    $$C_4(X_i, X_j, X_k, X_l) = E[\tilde{X}_i\tilde{X}_j\tilde{X}_k\tilde{X}_l] - C_2(\tilde{X}_i, \tilde{X}_j)C_2(\tilde{X}_k, \tilde{X}_l) - C_2(\tilde{X}_i, \tilde{X}_k)C_2(\tilde{X}_j, \tilde{X}_l) - C_2(\tilde{X}_i, \tilde{X}_l)C_2(\tilde{X}_j, \tilde{X}_k)$$
    This represents the **fourth-order interaction** among the variables. A non-zero value indicates **non-Gaussianity** and suggests complex interactions beyond second and third orders.

---

## Application of Cumulants

The script is designed to precisely calculate these cumulants (orders 2, 3, and 4) for **binarized EEG activity data** from synthetic patients, with a crucial focus on comparing interactions **during avalanches** versus **outside of them**.

The script works as follows:

1.  **Per-Patient Data Loading:** It iterates through each patient. For each patient, it loads the previously segmented binarized activity data (for "during avalanches" in `binarized_during_avalanches` and "outside avalanches" in `binarized_outside_avalanches`) from the `../results/avalanches/` folder.

2.  **Generalized Cumulant Calculation:** It uses dedicated functions (`compute_second_order_cumulant`, `compute_third_order_cumulant`, `compute_fourth_order_cumulant`) that implement the mathematical definitions mentioned above. These functions handle the combinations of channels to compute covariance, triplet, and quadruplet interactions, respectively, on the binarized data.

3.  **Segmented Analysis:** Cumulants are calculated separately for time periods **inside avalanches** and for periods **outside avalanches**. This allows for a direct comparison to understand how high-order interactions change based on the network's activity state.

4.  **Result Saving:** The calculated cumulants for each patient are stored in individual `.pkl` files (e.g., `patient_00_cumulants.pkl`) within the `../results/cumulants/` folder. This output format facilitates later analysis and aggregation of results at a group level.

In [ ]:
import numpy as np
import os
import pickle
from collections import defaultdict
import itertools

# --- Configuración ---
# Directorio donde se encuentran los datos segmentados por avalanchas de los pacientes.
# Este es el resultado del script anterior.
INPUT_AVALANCHE_DIR = "../results/avalanches"
# Directorio donde se guardarán los cumulantes calculados para cada paciente.
OUTPUT_CUMULANTS_DIR = "../results/cumulants"
# Número total de pacientes a procesar, debe coincidir con la generación de datos previa.
NUM_PATIENTS = 20

# --- Función Auxiliar: Cálculo de Cumulante de Segundo Orden ---
def compute_second_order_cumulant(X):
    """
    Calcula el cumulante de segundo orden (covarianza) para todos los pares únicos de canales.
    Es equivalente a la covarianza para datos centrados en la media.
    """
    n_channels = X.shape[1]
    cumulants = defaultdict(float)
    # Itera sobre pares únicos (i, j) donde i < j
    for i in range(n_channels):
        for j in range(i, n_channels): # Incluye i=j para la varianza (elementos diagonales)
            if i == j: # Varianza (elementos diagonales)
                cumulants[(i, j)] = np.var(X[:, i])
            else: # Covarianza (elementos fuera de la diagonal)
                cumulants[(i, j)] = np.cov(X[:, i], X[:, j])[0, 1]
    return cumulants

# --- Función Auxiliar: Cálculo de Cumulante de Tercer Orden ---
def compute_third_order_cumulant(X):
    """
    Calcula el cumulante de tercer orden para todas las tripletas únicas de canales.
    Esto captura interacciones de tripletas y está relacionado con el sesgo (skewness).
    Para variables aleatorias centradas en la media x, y, z, es E[xyz].
    """
    n_channels = X.shape[1]
    cumulants = defaultdict(float)
    if n_channels < 3:
        return cumulants # No se puede calcular el cumulante de 3er orden para menos de 3 canales

    # Centrar los datos en la media una vez para mayor eficiencia
    X_centered = X - np.mean(X, axis=0)

    for comb in itertools.combinations(range(n_channels), 3):
        i, j, k = comb
        val = np.mean(X_centered[:, i] * X_centered[:, j] * X_centered[:, k])
        cumulants[comb] = val
    return cumulants

# --- Función Auxiliar: Cálculo de Cumulante de Cuarto Orden ---
def compute_fourth_order_cumulant(X):
    """
    Calcula el cumulante de cuarto orden para todas las cuádruples únicas de canales.
    Esto captura interacciones de cuádruples y está relacionado con la curtosis (kurtosis).
    Para variables aleatorias centradas en la media x1, x2, x3, x4, es
    E[x1x2x3x4] - E[x1x2]E[x3x4] - E[x1x3]E[x2x4] - E[x1x4]E[x2x3].
    """
    n_channels = X.shape[1]
    cumulants = defaultdict(float)
    if n_channels < 4:
        return cumulants # No se puede calcular el cumulante de 4º orden para menos de 4 canales

    # Centrar los datos en la media una vez para mayor eficiencia
    X_centered = X - np.mean(X, axis=0)

    for comb in itertools.combinations(range(n_channels), 4):
        i, j, k, l = comb
        x1 = X_centered[:, i]
        x2 = X_centered[:, j]
        x3 = X_centered[:, k]
        x4 = X_centered[:, l]

        term1 = np.mean(x1 * x2 * x3 * x4)
        term2 = np.mean(x1 * x2) * np.mean(x3 * x4)
        term3 = np.mean(x1 * x3) * np.mean(x2 * x4)
        term4 = np.mean(x1 * x4) * np.mean(x2 * x3)

        val = term1 - term2 - term3 - term4
        cumulants[comb] = val
    return cumulants

# --- Script Principal para Calcular Cumulantes por Estado de Avalancha para Cada Paciente ---
if __name__ == "__main__":
    # Asegúrate de que el directorio de resultados de cumulantes exista.
    os.makedirs(OUTPUT_CUMULANTS_DIR, exist_ok=True)
    print(f"Directorio de salida '{OUTPUT_CUMULANTS_DIR}' asegurado.")

    print(f"Iniciando el cálculo de cumulantes para {NUM_PATIENTS} pacientes...")

    # Itera sobre cada paciente.
    for patient_idx in range(NUM_PATIENTS):
        # Construye el nombre del archivo para los datos de avalancha pre-segmentados del paciente actual.
        patient_avalanche_filename = os.path.join(INPUT_AVALANCHE_DIR, f"patient_{patient_idx:02d}_avalanches.pkl")

        # Verifica si el archivo de datos segmentados existe. Si no, advierte y salta a este paciente.
        if not os.path.exists(patient_avalanche_filename):
            print(f"Advertencia: Datos de avalancha segmentados no encontrados para el paciente {patient_idx:02d} en {patient_avalanche_filename}. Saltando.")
            continue
        
        print(f"\n--- Procesando Paciente {patient_idx:02d} para Cumulantes ---")
        
        with open(patient_avalanche_filename, 'rb') as f:
            patient_data = pickle.load(f)
        
        in_aval = patient_data['binarized_during_avalanches']
        out_aval = patient_data['binarized_outside_avalanches']
        
        # --- LÍNEA AJUSTADA AQUÍ ---
        # Obtiene el número de canales directamente de la forma de los datos binarizados.
        # Esto es más robusto ya que 'n_channels' no siempre podría estar en 'original_patient_params'.
        # Asumimos que 'in_aval' (o 'out_aval' como respaldo) siempre tendrá el número correcto de canales.
        if in_aval.shape[1] > 0:
            n_channels = in_aval.shape[1]
        elif out_aval.shape[1] > 0:
            n_channels = out_aval.shape[1]
        else: # Si ambos están vacíos en canales, no podemos determinar n_channels.
            print(f"Error: No se pudo determinar el número de canales para el paciente {patient_idx:02d}. Ambos 'in_aval' y 'out_aval' están vacíos en dimensiones de canal.")
            continue # Salta este paciente

        print(f"  Muestras dentro de avalanchas: {in_aval.shape[0]}")
        print(f"  Muestras fuera de avalanchas: {out_aval.shape[0]}")

        # Diccionario para almacenar los resultados de los cumulantes para el paciente actual.
        cumulants_results = {
            'in_aval': defaultdict(dict), # Para almacenar cumulantes 'dentro de avalanchas'
            'out_aval': defaultdict(dict) # Para almacenar cumulantes 'fuera de avalanchas'
        }

        # Lista de órdenes de cumulantes a calcular.
        orders_to_compute = [2, 3, 4]
        
        for order in orders_to_compute:
            print(f"  Calculando cumulantes de orden {order}...")
            
            # --- Calcular para 'Dentro de Avalanchas' ---
            # Se necesita un mínimo de 'orden + 1' muestras y 'orden' canales para calcular un cumulante de orden 'orden'.
            min_samples_needed = order + 1
            if in_aval.shape[0] >= min_samples_needed and n_channels >= order:
                if order == 2:
                    cumulants_results['in_aval'][order] = compute_second_order_cumulant(in_aval)
                elif order == 3:
                    cumulants_results['in_aval'][order] = compute_third_order_cumulant(in_aval)
                elif order == 4:
                    cumulants_results['in_aval'][order] = compute_fourth_order_cumulant(in_aval)
                print(f"    Cumulantes de orden {order} 'in_aval' calculados para {len(cumulants_results['in_aval'][order])} combinaciones.")
            else:
                print(f"    Saltando cumulantes de orden {order} 'in_aval' por datos insuficientes (muestras: {in_aval.shape[0]}, canales: {n_channels}). Se requieren al menos {min_samples_needed} muestras y {order} canales.")
                
            # --- Calcular para 'Fuera de Avalanchas' ---
            if out_aval.shape[0] >= min_samples_needed and n_channels >= order:
                if order == 2:
                    cumulants_results['out_aval'][order] = compute_second_order_cumulant(out_aval)
                elif order == 3:
                    cumulants_results['out_aval'][order] = compute_third_order_cumulant(out_aval)
                elif order == 4:
                    cumulants_results['out_aval'][order] = compute_fourth_order_cumulant(out_aval)
                print(f"    Cumulantes de orden {order} 'out_aval' calculados para {len(cumulants_results['out_aval'][order])} combinaciones.")
            else:
                print(f"    Saltando cumulantes de orden {order} 'out_aval' por datos insuficientes (muestras: {out_aval.shape[0]}, canales: {n_channels}). Se requieren al menos {min_samples_needed} muestras y {order} canales.")

        # Guarda los cumulantes calculados para el paciente actual.
        output_filename = os.path.join(OUTPUT_CUMULANTS_DIR, f"patient_{patient_idx:02d}_cumulants.pkl")
        with open(output_filename, 'wb') as f:
            pickle.dump(cumulants_results, f)
            
        print(f"  Resultados de cumulantes para el paciente {patient_idx:02d} guardados en '{output_filename}'")

    print("\nTodos los pacientes han sido procesados exitosamente para el cálculo de cumulantes.")
    print(f"Los resultados se encuentran en '{OUTPUT_CUMULANTS_DIR}'.")

Directorio de salida '../results/cumulants' asegurado.
Iniciando el cálculo de cumulantes para 20 pacientes...

--- Procesando Paciente 00 para Cumulantes ---
  Muestras dentro de avalanchas: 80
  Muestras fuera de avalanchas: 80
  Calculando cumulantes de orden 2...
    Cumulantes de orden 2 'in_aval' calculados para 154168020 combinaciones.
    Cumulantes de orden 2 'out_aval' calculados para 2980461 combinaciones.
  Calculando cumulantes de orden 3...
